In [1]:
import sys
sys.path.append('../../mist-base/GW')
sys.path.append('../../mist-base/')
sys.path.append('../../mist-base/utils')

import gw150814_simulator as gs
from gw150814_simulator import GW150814, defaults, GW150814_Additive
# import module

import torch
import numpy as np
import scipy
import scipy.stats
import pytorch_lightning as pl
from collections import defaultdict
from tqdm import tqdm
import plotfancy as pf
pf.housestyle_rcparams()

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import os, sys

from simulators.additive import Simulator_Additive
from simulators.utils import *
from utils.data import OnTheFlyDataModule, StoredDataModule
from utils.module import CustomLossModule_withBounds, BCELossModule

mycolors = ['#77aca2', '#ff004f', '#f98e08']
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')


In [2]:
default = gs.defaults
default['posterior_samples_path'] = '../../mist-base/GW/GW150814_posterior_samples.npz'
default['f_max']=250
gw = gs.GW150814(settings=default)
l = len(gw.generate_time_domain_waveform())
s = 1000
tdw = np.zeros([s,l])
tdn = np.zeros([s,l])

for i in tqdm(range(s)):
    tdw[i] = gw.generate_time_domain_waveform()
    tdn[i] = gw.generate_time_domain_noise()

Loading data for H1 at GPS time 1126259462.4
Loading PSD for H1 at GPS time 1126259462.4 [can take O(1 min)]
Loading posterior samples from ../../mist-base/GW/GW150814_posterior_samples.npz


100%|██████████████████████████████████████| 1000/1000 [00:03<00:00, 297.92it/s]


In [3]:
correlation_scales = torch.tensor([5]).int()
gw150814_post = torch.tensor(tdw)
gw150814_noise = torch.tensor(tdn)
gw150814_samples = {'mu': gw150814_post, 'noise': gw150814_noise}
simulator = GW150814_Additive(
    gw150814_samples=gw150814_samples, 
    bounds=torch.tensor([1.05]), #1.2341, 0.5696, 0.3403]), 
    dtype=torch.float32,
    correlation_scales = correlation_scales
)
times = simulator._process(simulator.times)
Nbins = simulator.Nbins

Loading data for H1 at GPS time 1126259462.4
Loading PSD for H1 at GPS time 1126259462.4 [can take O(1 min)]
Loading posterior samples from ../../mist-base/GW/GW150814_posterior_samples.npz


In [4]:
samples = simulator.sample(500)

# mu samples
plt.figure(figsize=(7, 3))
plt.title("Posterior samples")
for i in tqdm(range(500)):
    plt.plot(times, samples['mu'][i], color="C0", alpha=0.05)
plt.plot(simulator.times, simulator.filter_gwosc_data()*1e21, color="C1", zorder=10, label="GWOSC data")
plt.xlim(-0.1, 0.1)
plt.xlabel("Time [s]")
plt.ylabel("Strain")
plt.legend();

# x0 samples
plt.figure(figsize=(7, 3))
plt.title("Posterior samples + PSD")
for i in tqdm(range(500)):
    plt.plot(times, samples['x0'][i], color="C0", alpha=0.05)
plt.plot(simulator.times, simulator.filter_gwosc_data()*1e21, color="C1", zorder=10, label="GWOSC data")
plt.xlim(-0.1, 0.1)
plt.xlabel("Time [s]")
plt.ylabel("Strain")
plt.legend();

# xi samples
plt.figure(figsize=(7, 3))
plt.title("Posterior samples + PSD + distortions")
for i in tqdm(range(500)):
    plt.plot(times, samples['xi'][i, 0], color="C0", alpha=0.05)
plt.plot(simulator.times, simulator.filter_gwosc_data()*1e21, color="C1", zorder=10, label="GWOSC data")
plt.xlim(-0.1, 0.1)
plt.xlabel("Time [s]")
plt.ylabel("Strain")
plt.legend();

RuntimeError: The size of tensor a (16384) must match the size of tensor b (1000) at non-singleton dimension 1

In [5]:
mu = x0 = simulator.get_mu()
x0 = simulator.get_x_H0(mu)
ni = simulator.get_ni(x0)

In [8]:
shape(simulator.gw150814_samples['mu'])

NameError: name 'shape' is not defined

In [22]:
len(tdn)

16384